# RAG-LLMs

## Setup

### Imports

In [1]:
# imports
import os
from dotenv import load_dotenv
from ast import literal_eval
from langchain.chat_models import AzureChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain

from src.utils import *
from src.data_utils import SQLDBManager, Neo4jGraphManager
from sql_pgvector.chain_utils import RAGChainManager

%load_ext autoreload
%autoreload 2

### Config

In [2]:
# load env vars
host, port, db, user, password = load_postgres_env_variables()
# dbm = SQLDBManager.from_env()   # instantiate SQLDBManager
# setup_azure_openai()    # setup azure openai AD token
# print(f"AD token set: {os.environ['AZURE_OPENAI_AD_TOKEN']}")
table = 'pegadata.ppm_work_filtered'

## Prepare & init DB

In [7]:
# set db variables
# src_table = 'pegadata.ppm_work'
# req_cols_path = 'data/pega-as-clone/req_fields.txt'
# data_table = f'{src_table}_filtered'
# primary_key = 'pxinsname'
# text_col = 'pydescription'
# instantiate sql db manager & connect to sql db
# dbm = SQLDBManager.from_env()

# filter 'ppm_work' table and create new (if not exists) 
# sqldb_manager.filter_table(src_table, req_cols_path, primary_key)
# clean text in 'pydescription' before creating embs
# sqldb_manager.clean_html(data_table, [text_col], primary_key)

# copy ppm_work_filtered table to 'public' schema
# copy_table_query = f'''CREATE TABLE public.ppm_work_filtered AS
# SELECT * FROM pegadata.ppm_work_filtered;'''
# dbm.db.run(copy_table_query)

# create embeddings
# embs_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002") # instantiate embeddings model
# sqldb_manager.create_embs_col(data_table, text_col, embs_model) # create embeddings col

connected to database successfully.


''

In [3]:
# instantiate llm & db chain
# dbm = SQLDBManager.from_env()
# llm = AzureChatOpenAI(model='gpt-35-turbo', max_tokens=500, temperature=0.9)
# db_chain = SQLDatabaseChain.from_llm(llm, dbm.db, verbose=True)

connected to database successfully.


## RAG

In [23]:
# instantiate embs model, llm, and rag chain manager
embs_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002") # instantiate embeddings model
llm = AzureChatOpenAI(model='gpt-35-turbo', max_tokens=500, temperature=0.9, model_kwargs={"stop":["\nSQLResult:"]})  # instantiate lm
rcm = RAGChainManager(dbm.db, llm, embs_model)   # instantiate rag chain manager

In [25]:
# generate sql query
# user_query = "how many user stories are under epic 4?"
# user_query = "find the most recent user story under epic 4 and summarize it"
# user_query = "list all the user stories under epic 4, along w/ their priority (if known) and status"
user_query = "list all user stories with deadlines before Feb 1, 2024, along w their assignees and status"
# sql_query = rcm.gen_query(user_query)
# print(f"query: {user_query}\ngenerated sql query: {sql_query}") 

# generate response using rag
sql_query, response = rcm.gen_response(user_query)
query_res = dbm.db.run(sql_query)   # run generated sql query on db 
print(f"query: {user_query}\n\nsql query: {sql_query}\n\nsql query result: {query_res}\n\nresponse: {response}")

query: list all user stories with deadlines before Feb 1, 2024, along w their assignees and status

sql query: SELECT "pxinsname", "pyoriguserid", "pystatuswork" FROM "ppm_work_filtered" WHERE "pxobjclass" = 'PegaProjMgmt-Work-UserStory' AND "pysladeadline" < date '2024-02-01' LIMIT 5

sql query result: [('US-4', 'Administrator', 'Pending-Details'), ('US-6', 'Administrator', 'Pending-Details'), ('US-9', 'Administrator', 'Pending-Details'), ('US-1', 'Administrator', 'Pending-Details'), ('US-10', 'Administrator', 'Pending-Details')]

response: The SQL query is selecting the values of "pxinsname", "pyoriguserid", and "pystatuswork" from the "ppm_work_filtered" table. It is filtering the results based on the condition that "pxobjclass" is equal to 'PegaProjMgmt-Work-UserStory' and "pysladeadline" is less than the date '2024-02-01'. The query is limited to returning only 5 rows. The SQL response shows the resulting values for the specified columns: [('US-4', 'Administrator', 'Pending-Detail

## KG-RAG

### Create neo4j graph from postgres db

In [4]:
ngm = Neo4jGraphManager.from_env()  # instantiate neo4j graph manager
ngm.from_table(table, reset=True)    # create graph from table

g:\My Drive\intern@Pega\rag-llm\src\data_utils.py:245: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
